In [24]:
from binance.client import Client
import pandas as pd
import pandas_ta as ta
from prophet import Prophet
import logging
logging.getLogger("cmdstanpy").setLevel(logging.ERROR)

In [2]:
# Initialize the Binance client without authentication
client = Client()


In [3]:
from datetime import datetime, timedelta

# Fetch the last 40 records of the 5-minute candle for BTCUSDT
symbol = 'BTCUSDT'
interval = '1m'  # Valid interval for 5-minute candles
limit = 60

# Calculate the timestamp for one year ago
one_year_ago = datetime.now() - timedelta(days=365)
one_year_ago_timestamp = int(one_year_ago.timestamp() * 1000)

# Fetch klines data from one year ago until now
klines = []
while True:
    new_klines = client.get_klines(symbol=symbol, interval=interval, startTime=one_year_ago_timestamp, limit=1000)
    if not new_klines:
        break
    klines.extend(new_klines)
    one_year_ago_timestamp = new_klines[-1][0] + 1  # Move to the next timestamp

# Ensure we have data up to the current time
df = klines
klines = klines[:limit]

In [4]:
bkp_df = df

In [5]:
# Convert the data to a pandas DataFrame
columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
df = pd.DataFrame(bkp_df, columns=columns)

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Convert 'high', 'low', 'close', 'volume' columns to decimals
df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].apply(pd.to_numeric)
df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

df.head(100)

,Open,High,Low,Close,Volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
timestamp,,,,,,,,,,,
2024-03-21 06:16:00,67267.77,67289.12,67194.21,67194.22,39.42142,1711001819999,2651562.33176900,1195,20.50866000,1379606.75848560,0
2024-03-21 06:17:00,67194.21,67211.75,67101.10,67101.10,33.68029,1711001879999,2262234.97182050,1247,7.90357000,530874.99776200,0
2024-03-21 06:18:00,67101.11,67150.00,67064.83,67101.45,32.88355,1711001939999,2206668.31252670,1436,15.37712000,1031903.15959040,0
2024-03-21 06:19:00,67101.45,67108.00,67050.17,67074.54,33.32157,1711001999999,2234980.61042810,1090,15.70272000,1053147.51437700,0
2024-03-21 06:20:00,67074.54,67074.54,67050.17,67061.78,18.56301,1711002059999,1244845.27185430,759,10.31279000,691565.72440930,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-03-21 07:51:00,67035.64,67035.64,66940.92,66969.99,30.41046,1711007519999,2037294.60878020,1155,6.31405000,422804.29277570,0
2024-03-21 07:52:00,66969.99,66983.68,66944.20,66944.21,22.41500,1711007579999,1500973.02352370,949,13.04013000,873183.57792280,0
2024-03-21 07:53:00,66944.21,66974.38,66944.20,66958.65,18.23212,1711007639999,1220799.51464260,1050,7.16904000,480021.63976790,0


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG
from tqdm import tqdm


capped_history = df.iloc[-1000000:]

SL_FACTOR = 0.1
TP_FACTOR = 0.2
BUY_THRESHOLD = 0.04
SELL_THRESHOLD = -0.04


ROWS_AHEAD = 5
LOOK_BACK = 30


class ProphetStrategy(Strategy):
    def init(self):
        pass

    def next(self):

        if len(self.data) < LOOK_BACK:
            return

        rolling_training_data = pd.DataFrame(
            {
                "Volume": self.data.Volume[-LOOK_BACK:],
                "Open": self.data.Open[-LOOK_BACK:],
                "High": self.data.High[-LOOK_BACK:],
                "Low": self.data.Low[-LOOK_BACK:],
                "number_of_trades": self.data.number_of_trades[-LOOK_BACK:],
                "taker_buy_base_asset_volume": self.data.taker_buy_base_asset_volume[
                    -LOOK_BACK:
                ],
                "ds": self.data.index[-LOOK_BACK:],
                "y": self.data.Close[-LOOK_BACK:],
            }
        )
        # Create and fit model
        model = Prophet(
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10
        )
        # model.add_regressor('Volume')
        # model.add_regressor('Open')
        # model.add_regressor('High')
        # model.add_regressor('Low')
        # model.add_regressor('number_of_trades')
        # model.add_regressor('taker_buy_base_asset_volume')

        model.fit(rolling_training_data)

        future = model.make_future_dataframe(periods=ROWS_AHEAD, freq="5min")
        forecast = model.predict(future)

        # Store only the forecasted values (last FORECAST_STEPS rows)
        predictions = forecast.iloc[-ROWS_AHEAD:]["yhat"].values
        dates = forecast.iloc[-ROWS_AHEAD:]["ds"].values

        max_prediction = (max(predictions) / self.data.Close[-1] ) - 1
        min_prediction = (min(predictions) / self.data.Close[-1] ) - 1

        if(max_prediction > BUY_THRESHOLD):
            BUY_SIGNAL = True
        else:
            BUY_SIGNAL = False

        if(min_prediction < SELL_THRESHOLD):
            SELL_SIGNAL = True
        else:
            SELL_SIGNAL = False

        if(len(self.orders) > 0):
            for order in self.orders:
                if order.is_contingent == 0:
                    order.cancel()

        if BUY_SIGNAL and self.position.size < 0:
            self.position.close()
            # print("a Close sell position")

        if SELL_SIGNAL and self.position.size > 0:
            self.position.close()

        if self.position.size == 0:
            if BUY_SIGNAL:
                # print('Buy signal')

                tp_raw_price = self.data.Close[-1]+(self.data.Close[-1]*TP_FACTOR)
                sl_raw_price = self.data.Close[-1]-(self.data.Close[-1]*SL_FACTOR)

                self.buy(size=1, sl=sl_raw_price, tp=tp_raw_price)
                # self.buy(size=1)
            elif SELL_SIGNAL:

                tp_raw_price = self.data.Close[-1]-(self.data.Close[-1]*TP_FACTOR)
                sl_raw_price = self.data.Close[-1]+(self.data.Close[-1]*SL_FACTOR)

                self.sell(size=1, sl=sl_raw_price, tp=tp_raw_price)
                # self.sell(size=1)


# Run backtest
bt = Backtest(capped_history, ProphetStrategy, cash=1000000, commission=0.002)
stats = bt.run()
bt.plot()
stats